In [ ]:
## Open AI API Key and Open Source models -- Llama3,Gemma2,mistral--groq
import os
from dotenv import load_dotenv
load_dotenv()

# load  openai api key
import openai
openai.api_key=os.getenv("OPENAI_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")

In [ ]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model
## now with help of this application we are going to create our whole application


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000186CF023310>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000186CF020E50>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
## whenever i want to interact, i.e user gives query , buliding llm application basically means if i m giving query to my llm model , my model should be able to give me some response
## but we should understand some key libtraries that r specifically used in langchain which we also see it as runnables 
# common libraries : langchain_core
from langchain_core.messages import HumanMessage,SystemMessage
## why these system and human messages are important . whenever we provide any message to our llm model , we should specicy to our llm model which message is  provided by human being and which mesaage is kind of an instruction to llm model
messages=[
    SystemMessage(content="Translate the following from english to French"),
    HumanMessage(content="Hello how are you?")
]

result=model.invoke(messages)
result

AIMessage(content='Bonjour, comment allez-vous ? \n\n\nYou can also use:\n\n* **Salut, comment vas-tu ?**  (informal, used with friends)\n* **Coucou, comment allez-vous ?** (informal, more casual)\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 21, 'total_tokens': 76, 'completion_time': 0.1, 'prompt_time': 0.001322239, 'queue_time': 0.247433051, 'total_time': 0.101322239}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c8a061df-0c25-4988-9e0f-c5a68e2dab18-0', usage_metadata={'input_tokens': 21, 'output_tokens': 55, 'total_tokens': 76})

In [8]:
## we want to retrieve op from results with help of output parser which will dispaly message coming as op of llm model
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Bonjour, comment allez-vous ? \n\n\nYou can also use:\n\n* **Salut, comment vas-tu ?**  (informal, used with friends)\n* **Coucou, comment allez-vous ?** (informal, more casual)\n'

In [9]:
## there is also somethimng called as lcel which help in chaining components
## using LCEL
chain=model|parser
chain.invoke(messages)

'Bonjour, comment allez-vous ? \n\n\nThis is the most common and formal way to say "Hello, how are you?" in French. \n\nHere are some other options:\n\n*  **Salut, ça va ?** (Informal, used with friends or family)\n* **Coucou, comment vas-tu ?** (Very informal, used with close friends)\n* **Bonjour, comment allez-vous ?** (Formal, used with strangers or people you respect)\n\n\n\nLet me know if you have any other phrases you\'d like translated!\n'

In [10]:
##  instead of passing message we can u another eficient technique known as prompt templates
from langchain_core.prompts import ChatPromptTemplate
generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages([
    ("system",generic_template),
    ("user","{text}")
    ])


In [12]:
result=prompt.invoke({"language":"French","text":"hello"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hello', additional_kwargs={}, response_metadata={})])

In [13]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hello', additional_kwargs={}, response_metadata={})]

In [16]:
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello sir"})

'Bonjour monsieur. \n'

In [ ]:
## Langserve Helps developers Deploy Langchain runnable and Chains as a REST API's
## langservew is integrated with fast api